In [46]:
import requests
import pandas as pd
import numpy as np
import time

In [3]:
def similarweb(website):
	url = "https://similarweb2.p.rapidapi.com/trafficoverview"
	querystring = {"website":"https://www."+website}

	headers = {
		"X-RapidAPI-Host": "similarweb2.p.rapidapi.com",
		"X-RapidAPI-Key": "b00f8a11c7msh7cb123610208633p1d2f21jsn969dea2cff92"
	}

	return requests.request("GET", url, headers=headers, params=querystring)

In [4]:
resp = similarweb('1tv.ru')

In [5]:
resp.content

b'{"engagement":{"avgVisitDuration":188,"bounceRate":0.5693,"pagesPerVisit":2.28,"totalVisits":27128000},"monthlyVisitsEstimate":{"2021-10-01":22180000,"2021-11-01":23410000,"2021-12-01":27000000,"2022-01-01":21320000,"2022-02-01":38180000,"2022-03-01":27120000},"name":"1tv.ru","trafficShareByCountry":[{"Russian Federation":0.8135},{"Ukraine":0.0579},{"Belarus":0.0245},{"Germany":0.0183},{"United States":0.0161}],"trafficSources":{"Direct":0.4514,"Mail":0.0058,"Paid Referrals":0.0001,"Referrals":0.0806,"Search":0.441,"Social":0.0209}}\n'

In [8]:
ls

similar_web.ipynb
Анализ медиа пространства_202011.xlsx
Анализ медиа пространства_20220504.xlsx


In [27]:
# getting the list of websites
df_sources = pd.read_excel('Анализ медиа пространства_20220504.xlsx', header=1)
sites = df_sources['source'].to_list()

In [60]:
# collecting info
# sites = ['echo.msk.ru'] # не подгрузившиеся сайты
df_traffic_all = []
for website in sites:
    # trying to get and read k times. if error try again.
    k = 3
    while k>0:
        try:
            response = similarweb(website)
            df_traffic = pd.DataFrame({website:list(response.json()['monthlyVisitsEstimate'].values())}).T #often does not return proper repsonse (like not enough data)
            df_traffic.columns = list(response.json()['monthlyVisitsEstimate'].keys())
            k = 0
        except:
            k -= 1
    df_traffic_all.append(df_traffic)
    time. sleep(1)

In [51]:
similarweb(website).content

b'{"engagement":{"avgVisitDuration":96,"bounceRate":0.7116,"pagesPerVisit":1.76,"totalVisits":12570000},"monthlyVisitsEstimate":{"2021-10-01":6973000,"2021-11-01":6608000,"2021-12-01":7132000,"2022-01-01":7561000,"2022-02-01":8360000,"2022-03-01":12570000},"name":"forbes.ru","trafficShareByCountry":[{"Russian Federation":0.8154},{"Ukraine":0.0463},{"Kazakhstan":0.0252},{"Belarus":0.0245},{"United States":0.0137}],"trafficSources":{"Direct":0.5367,"Mail":0.0035,"Paid Referrals":0.0014,"Referrals":0.1091,"Search":0.2974,"Social":0.0515}}\n'

In [61]:
(pd.concat(df_traffic_all)/1000000).to_clipboard(excel=True)

In [ ]:
param = 'trafficShareByCountry'
type(response.json()[param]), response.json()[param]

(list,
 [{'Russian Federation': 0.5112},
  {'United States': 0.0911},
  {'Ukraine': 0.0774},
  {'Germany': 0.0417},
  {'Netherlands': 0.0293}])

In [ ]:
param = 'engagement'
type(response.json()[param]), response.json()[param]

(dict,
 {'avgVisitDuration': 170,
  'bounceRate': 0.6133,
  'pagesPerVisit': 2.26,
  'totalVisits': 207820000})